In [1]:
import time
import requests

from jupyter_dash import JupyterDash
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html

from dash.dependencies import Output
from dash.dependencies import Input

In [2]:
IP = '192.168.1.11'
EXTERNAL_STYLESHEETS = ['https://cdn.jsdelivr.net/npm/bootswatch@4.5.2/dist/minty/bootstrap.min.css']
REFRESH_TIME = 3000

In [3]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.MINTY])
server = app.server

In [4]:
def create_card(title='', width='100%'):
    return dbc.Card(
        [
            dbc.CardBody([
                html.H4(title, className='card-title'),
                html.P('-', className='card-text', id='{}_card'.format(title.lower())),
            ]),
        ],
        style={'width': width},
    )

In [5]:
app.layout = html.Div([
    dcc.Interval(id='interval', interval=REFRESH_TIME),
    dbc.Row([
        create_card(title='Temperature', width='20%'),
        create_card(title='Humidity', width='20%'),
        create_card(title='Moisture', width='20%'),
    ]),
])

In [6]:
@app.callback(
    Output('temperature_card', 'children'),
    Output('humidity_card', 'children'),
    Output('moisture_card', 'children'),
    Input('interval', 'n_intervals')
)
def read_sensors(n_clicks):
    sensors_data = requests.get('http://{}/read'.format(IP))
    
    if sensors_data.status_code != 200:
        return ['-'] * 3
    
    values = sensors_data.json()
    return values['temperature'], values['humidity'], values['moisture']

In [7]:
app.run_server(mode='inline', debug=True)

In [8]:
app.run_server()

Dash app running on http://127.0.0.1:8050/
